## Titanic Dataset

In [1]:
import datalabframework as dlf

In [2]:
dlf.project.rootpath()

'/home/jovyan/src'

## Data

In [3]:
metadata = dlf.params.metadata()
metadata

['/home/jovyan/src/metadata.yml', '/home/jovyan/src/etl/raw/metadata.yml', '/home/jovyan/src/etl/extract/metadata.yml', '/home/jovyan/src/etl/clean/metadata.yml', '/home/jovyan/src/ml/features/metadata.yml']


{'engines': {'spark': {'context': 'spark',
   'config': {'master': 'local[4]', 'jobname': 'test'}},
  'pandas': {'context': 'pandas'},
  'numpy': {'context': 'numpy'}},
 'loggers': {'stream': {'enable': True, 'severity': 'info'},
  'kafka': {'enable': False,
   'severity': 'info',
   'topic': 'dlf',
   'hosts': ['localhost:29092']}},
 'providers': {'local': {'service': 'fs', 'rootpath': '../data'}},
 'resources': {'.etl.extract.train': {'path': 'datasets/extract/train',
   'format': 'parquet',
   'provider': 'local'},
  '.etl.clean.train': {'path': 'datasets/clean/train',
   'format': 'parquet',
   'provider': 'local'},
  '.ml.features.test': {'path': 'datasets/features/test',
   'format': 'parquet',
   'provider': 'local'},
  '.ml.features.train': {'path': 'datasets/features/train',
   'format': 'parquet',
   'provider': 'local'},
  '.etl.raw.test': {'path': 'datasets/raw/test.csv',
   'format': 'csv',
   'provider': 'local'},
  '.etl.clean.test': {'path': 'datasets/clean/test',
   'f

#### Init Spark

In [4]:
engine = dlf.engines.get('spark')
spark = engine.context()

['/home/jovyan/src/metadata.yml', '/home/jovyan/src/etl/raw/metadata.yml', '/home/jovyan/src/etl/extract/metadata.yml', '/home/jovyan/src/etl/clean/metadata.yml', '/home/jovyan/src/ml/features/metadata.yml']


In [27]:
#print out name and version
'{}:{}'.format(engine.info['context'], spark.sparkSession.version)

'spark:2.3.0'

## Train

#### Load Dataset

In [28]:
df = engine.read('.etl.raw.train', header=True, inferSchema=True)
for column in df.schema:
    print('{:<12} {:<10} {}'.format(column.name, str(column.dataType)[:-4], column.nullable))

PassengerId  Integer    True
Survived     Integer    True
Pclass       Integer    True
Name         String     True
Sex          String     True
Age          Double     True
SibSp        Integer    True
Parch        Integer    True
Ticket       String     True
Fare         Double     True
Cabin        String     True
Embarked     String     True


#### Save as parquet

In [29]:
engine.write(df, '.etl.extract.train', mode='overwrite')

#### Explore the dataset

Check for Null or NaN values, and count them per column

In [30]:
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



## Test

#### Load Dataset

In [31]:
df = engine.read('.etl.raw.test', header=True, inferSchema=True)
for column in df.schema:
    print('{:<12} {:<10} {}'.format(column.name, str(column.dataType)[:-4], column.nullable))

PassengerId  Integer    True
Pclass       Integer    True
Name         String     True
Sex          String     True
Age          Double     True
SibSp        Integer    True
Parch        Integer    True
Ticket       String     True
Fare         Double     True
Cabin        String     True
Embarked     String     True


#### Save as parquet

In [32]:
engine.write(df,'.etl.extract.test', mode='overwrite')

#### Explore the dataset

In [33]:
for column in df.schema:
    print('{:<12} {:<10} {}'.format(column.name, str(column.dataType)[:-4], column.nullable))

PassengerId  Integer    True
Pclass       Integer    True
Name         String     True
Sex          String     True
Age          Double     True
SibSp        Integer    True
Parch        Integer    True
Ticket       String     True
Fare         Double     True
Cabin        String     True
Embarked     String     True


Check for Null or NaN values, and count them per column

In [34]:
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|     0|   0|  0| 86|    0|    0|     0|   1|  327|       0|
+-----------+------+----+---+---+-----+-----+------+----+-----+--------+

